In [25]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import ExtraTreesRegressor

In [26]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [27]:
y_train = train['y'].values
id_test = test['ID']


In [6]:
num_train = len(train)
df_all = pd.concat([train, test])
df_all.drop(['ID', 'y'], axis=1, inplace=True)

C:\Users\zeesh\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [8]:
train.head(5)

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [9]:
test.sample(5)

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
100,191,n,l,as,f,d,j,i,f,0,...,0,0,0,0,0,0,0,0,0,0
1485,2980,ak,c,as,a,d,ad,g,y,0,...,0,0,1,0,0,0,0,0,0,0
2990,5965,z,aa,ay,c,d,p,i,w,1,...,1,0,0,0,0,0,0,0,0,0
2607,5207,f,l,as,f,d,m,i,o,0,...,0,0,0,0,0,0,0,0,0,0
1052,2115,w,s,as,c,d,ag,i,a,0,...,1,0,0,0,0,0,0,0,0,0


In [10]:
# One-hot encoding of categorical/strings
df_all = pd.get_dummies(df_all, drop_first=True)

In [11]:
train = df_all[:num_train]
test = df_all[num_train:]

In [12]:
train.head(2)

,X10,X100,X101,X102,X103,X104,X105,X106,X107,X108,...,X8_p,X8_q,X8_r,X8_s,X8_t,X8_u,X8_v,X8_w,X8_x,X8_y
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
test.head(2)

,X10,X100,X101,X102,X103,X104,X105,X106,X107,X108,...,X8_p,X8_q,X8_r,X8_s,X8_t,X8_u,X8_v,X8_w,X8_x,X8_y
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [14]:
model = ExtraTreesRegressor(n_estimators=100, n_jobs=4, min_samples_split=25,
                            min_samples_leaf=35, max_features=150)

In [15]:
gsc = GridSearchCV(
    estimator=model,
    param_grid={
                #'n_estimators': range(50,126,25),
                'max_features': range(50,401,50),
                #'min_samples_leaf': range(20,50,5),
                #'min_samples_split': range(15,36,5),
                },
    scoring='r2',
    cv=5
)

In [16]:
grid_result = gsc.fit(train, y_train)

In [17]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.559139 using {'max_features': 150}


In [18]:
for test_mean, train_mean, param in zip(
                                        grid_result.cv_results_['mean_test_score'],
                                        grid_result.cv_results_['mean_train_score'],
                                        grid_result.cv_results_['params']):
    print("Train: %f // Test : %f with: %r" % (train_mean, test_mean, param))

Train: 0.580168 // Test : 0.552613 with: {'max_features': 50}
Train: 0.595477 // Test : 0.557907 with: {'max_features': 100}
Train: 0.601790 // Test : 0.559139 with: {'max_features': 150}
Train: 0.604667 // Test : 0.557643 with: {'max_features': 200}
Train: 0.606420 // Test : 0.557648 with: {'max_features': 250}
Train: 0.607208 // Test : 0.557289 with: {'max_features': 300}
Train: 0.607570 // Test : 0.555584 with: {'max_features': 350}
Train: 0.607601 // Test : 0.555369 with: {'max_features': 400}


C:\Users\zeesh\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


In [19]:
model = ExtraTreesRegressor(**grid_result.best_params_)

model.fit(train, y_train)

df_sub = pd.DataFrame({'ID': id_test, 'y': model.predict(test)})
df_sub.to_csv('mercedes-submission.csv', index=False)

C:\Users\zeesh\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [20]:
df_sub=pd.read_csv("mercedes-submission.csv",header=0)

In [28]:
df_sub.head(4)

,ID,y
0,1,77.0450
1,2,96.8750
2,3,76.3405
3,4,75.9820
